In [1]:
import os
import json
import pandas as pd
import subprocess

# Function to clean and format file names and extract numeric part
def clean_file_name(file_name):
    base_name = os.path.splitext(file_name)[0]  # Remove extension
    cleaned_name = base_name.replace('_', ' ').replace('code full', '').strip()
    return cleaned_name

def extract_numeric_part(file_name):
    parts = file_name.split()
    for part in parts:
        if part.isdigit():
            return int(part)
    return 0

In [2]:
fidelity_py = "../simulator.py"
data_folder = "../"
os.makedirs(data_folder,exist_ok=True)
data_folder = f"{data_folder}/"
data_folder

'..//'

In [3]:
# Directory containing the JSON files
folder_map = False
code_folder_path = f'../linear_map'
fid_folder_path = f'../fid/'

In [4]:

for file_name in os.listdir(code_folder_path):
    # Construct the full path to the code file
    code_file_path = os.path.join(code_folder_path, file_name)
    if "json" in file_name:
        # Construct the command to run the simulator
        command = ["python", fidelity_py, code_file_path, fid_folder_path]
        # Execute the command
        subprocess.run(command)

In [5]:
# List to store the data
data_list = []

# Loop through files in the directory
for file_name in os.listdir(fid_folder_path):
    file_path = os.path.join(fid_folder_path, file_name)
    with open(file_path, 'r') as file:
        data = json.load(file)
        cleaned_name = clean_file_name(file_name)
        data['file_name'] = cleaned_name  # Clean and format the file name
        data['numeric_part'] = extract_numeric_part(cleaned_name)
        # print(cleaned_name)
        data_list.append(data)
# print(len(data_list))

# Create a DataFrame for the main metrics
df_main = pd.DataFrame([
    {'file_name': d['file_name'], **{k: v for k, v in d.items() if k != 'file_name' and isinstance(v, (int, float))}}
    for d in data_list
])

# Sort the DataFrame based on the numeric part
df_main = df_main.sort_values(by='numeric_part').drop(columns='numeric_part')

# Create separate DataFrames for the lists
movement_time_ratio_list = []
list_movement_duration_list = []

for d in data_list:
    for idx, value in enumerate(d['movement_time_ratio']):
        movement_time_ratio_list.append({
            'file_name': d['file_name'],
            'index': idx,
            'movement_time_ratio': value
        })
    for idx, value in enumerate(d['list_movement_duration']):
        list_movement_duration_list.append({
            'file_name': d['file_name'],
            'index': idx,
            'list_movement_duration': value
        })

df_movement_time_ratio = pd.DataFrame(movement_time_ratio_list)
df_list_movement_duration = pd.DataFrame(list_movement_duration_list)

# Display the DataFrames for analysis
print("Main DataFrame:")
print(df_main.head())

print("\nMovement Time Ratio DataFrame:")
print(df_movement_time_ratio.head())

print("\nList Movement Duration DataFrame:")
print(df_list_movement_duration.head())

# Save the DataFrames to CSV files if needed
df_main.to_csv(data_folder+f'{folder_map}_combined_main_results_.csv', index=False)
# df_movement_time_ratio.to_csv(data_folder+f'{map_type}_combined_movement_time_ratio.csv', index=False)
# df_list_movement_duration.to_csv(data_folder+f'{map_type}_combined_list_movement_duration.csv', index=False)

Main DataFrame:
                  file_name  cir_fidelity  cir_fidelity_1q_gate  \
0  qft 5 full code fidelity      0.762053                     1   
1  qft 6 full code fidelity      0.658886                     1   
2  qft 7 full code fidelity      0.552711                     1   
3  qft 8 full code fidelity      0.449829                     1   
4  qft 9 full code fidelity      0.355187                     1   

   cir_fidelity_2q_gate  cir_fidelity_2q_gate_for_idle  \
0              0.904610                       0.927656   
1              0.860384                       0.886787   
2              0.810157                       0.839273   
3              0.755253                       0.786392   
4              0.697047                       0.729501   

   cir_fidelity_atom_transfer  cir_fidelity_coherence  \
0                    0.924928                0.981812   
1                    0.888644                0.971786   
2                    0.846976                0.959742   
3   